# 爬取腾讯2019_nCoV中国疫情数据

## 0,环境准备

1，anaconda默认安装所有需要的库，通过import来验证，或使用pip安装requests，json，pandas，jupyter notebook

2，准备chrome或火狐浏览器

In [86]:
import requests
import json
import pandas as pd
requests.__version__, json.__version__, pd.__version__

('2.21.0', '2.0.9', '0.23.4')

## 1、分析数据源

1，通过浏览器打开数据源：https://news.qq.com/zt2020/page/feiyan.htm?from=timeline&isappinstalled=0

2，打开开发者选项，分析network流；

3，简述动态网页和静态网页的区别；

## 2、抓取数据

1，找到数据流，用python获取，查看内容；

2，用json加载数据，用可视化工具分析http://www.bejson.com/jsonviewernew/

In [ ]:
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery34105042688433769531_1581048970987&_=1581048970988'
reponse = requests.get(url=url)
data = reponse.text.split('jQuery34105042688433769531_1581048970987')[1][1:-1]
data = json.loads(data)
data

In [ ]:
data = json.loads(data['data'])
data

3，用json可视化工具，查看数据

4，封装函数：利用数据源得出的链接，获取到原始数据，然后用json处理成python字典并返回

In [89]:
def catch_data():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery34105042688433769531_1581048970987&_=1581048970988'
    reponse = requests.get(url=url)
    data = reponse.text.split('jQuery34105042688433769531_1581048970987')[1][1:-1]
    data = json.loads(data)
    if data['ret'] == 0:
        return json.loads(data['data'])

5，利用函数获取到到数据，看一下国内感染数据及今日新增感染情况

In [90]:
data = catch_data()
lastUpdateTime = data['lastUpdateTime']
chinaTotal = data['chinaTotal']
chinaAdd = data['chinaAdd']
chinaTotal

{'confirm': 37252, 'suspect': 28942, 'dead': 813, 'heal': 2747}

In [91]:
chinaAdd

{'confirm': 2654, 'suspect': 1285, 'dead': 90, 'heal': 695}

## 3、利用pandas清洗国内感染数据并保存到csv

In [92]:
# 定义数据处理函数，给定一个str，用eval变成字典，取其中元素并返回
def confirm(x):
    confirm = eval(str(x))['confirm']
    return confirm
def suspect(x):
    suspect = eval(str(x))['suspect']
    return suspect
def dead(x):
    dead = eval(str(x))['dead']
    return dead
def heal(x):
    heal =  eval(str(x))['heal']
    return heal

Pandas中文网：https://www.pypandas.cn/

In [97]:
#细致筛选数据，生成list
def extract_china(china_data):
    china_list = []
    for a in range(len(china_data)):
        province = china_data[a]['name']
        province_list = china_data[a]['children']
        for b in range(len(province_list)):
            city = province_list[b]['name']
            total = province_list[b]['total']
            today = province_list[b]['today']
            china_dict = {}
            china_dict['province'] = province
            china_dict['city'] = city
            china_dict['total'] = total
            china_dict['today'] = today
            china_list.append(china_dict)

    china_data = pd.DataFrame(china_list)
    # 函数映射
    china_data['confirm'] = china_data['total'].map(confirm)
#     china_data['suspect'] = china_data['total'].map(suspect)
    china_data['dead'] = china_data['total'].map(dead)
    china_data['heal'] = china_data['total'].map(heal)
    china_data['addconfirm'] = china_data['today'].map(confirm)
#     china_data['addsuspect'] = china_data['today'].map(suspect)
    china_data['adddead'] = china_data['today'].map(dead)
    china_data['addheal'] = china_data['today'].map(heal)
    china_data = china_data[["province","city","confirm","dead","heal","addconfirm"]]
    return china_data

利用上面的函数，原始数据中‘areaTree’中的‘children’清洗出来

In [98]:
areaTree = data['areaTree']
china_data = areaTree[0]['children']#0-中国
china_data = extract_china(china_data)
china_data.head()

,province,city,confirm,dead,heal,addconfirm
0,湖北,武汉,14982,608,877,1379
1,湖北,孝感,2436,29,45,123
2,湖北,黄冈,2141,43,135,100
3,湖北,荆州,997,13,40,56
4,湖北,襄阳,988,7,40,81


把清洗出的数据，存成csv。

In [103]:
china_data.to_csv('china_data.csv')

全国按天计数数据

In [102]:
chinaDayList = pd.DataFrame(data['chinaDayList'])
chinaDayList = chinaDayList[['date','confirm','suspect','dead','heal']]
chinaDayList.to_csv('china_daily_data.csv')
chinaDayList

,date,confirm,suspect,dead,heal
0,01.13,41,0,1,0
1,01.14,41,0,1,0
2,01.15,41,0,2,5
3,01.16,45,0,2,8
4,01.17,62,0,2,12
5,01.18,198,0,3,17
6,01.19,275,0,4,18
7,01.20,291,54,6,25
8,01.21,440,37,9,25
9,01.22,571,393,17,25


## 4*、数据可视化